# Weather Information Generator: OpenWeather API
## Author: Ahria Dominguez
### Last Updated: 6/29/2024

In this project, we will generate weather reports using the OpenWeather API source and user inputs of either zip codes or cities and states.

Please insert your own API key into the 'appid' variable below to use this generator.

#### Library Imports

In [1]:
# Imports the requests library to make API calls.
import requests

#### API Key

In [2]:
# Variable to insert your own API key. 
appid = ""

#### Generator Functions

In [3]:
# Initializes the generator by printing a welcome message and asking the user
# if they would like to use the weather service, allowing them to say yes (Y)
# or no (N). 
def main():
    print("Welcome to the United States OpenWeather Generator! Please keep in "
         "mind to only enter U.S. cities, states, or zip codes.")
    # Starts a loop to loop through all of the functions as many times as the
    # user would like until they break it by saying no to the following question.
    while True:
        retrieve_weather = input("\nWould you like to use the weather service? "
                                 "Y - Yes; N - No: ")
        # If the user says "y" or "Y", it will move onto the 
        # retrieve_user_info() function.
        if retrieve_weather.lower() == "y":
            try:
                retrieve_user_info()
            # Handles values other than "y" or "n".
            except ValueError:
                print("Please enter a 'Y' or an 'N'.")
            # Handles other, unexpected errors.
            else:
                print("I'm sorry. I didn't understand your response.")
        # If the user says "n" or "N", it will print out a thank you
        # message and break the loop.
        elif retrieve_weather.lower() == "n":
            print("Thanks for using the program!")
            break
        # Handles values other than "y" or "n".
        else:
            print("Please try again. Enter a 'Y' or an 'N'.")

In [4]:
# This is a function to allow the user to choose between a city or zip code.
def retrieve_user_info(): 
    # Loops through the following input until an acceptable input is received.
    while True:
        # Asks the user if they would like to use a city or zip code.
        zip_or_city = input("\nWould you like to enter a city name ('C') or zip"
                            "code ('Z')? ")
        # If the user says zip code (z/Z), it will move on to the zip_code_func()
        # function.
        if zip_or_city.lower() == "z":
            zip_code_func()
        # If the user says city (c/C), it will move onto the city_state_func()
        # function.
        elif zip_or_city.lower() == "c":
            city_state_func()
        # Handles inputs that are not "c", "C", "z", or "Z".
        else:
            print("Enter a 'C' for city or 'Z' for zip code. Try again.")

In [5]:
# This function is to call the city/state API link based on the user's city/state.
def city_state_func(): 
    # Loops through the following input until an acceptable response is received.
    while True:
        # Asks the user for the city and state name they wish to look up.
        city_name = input("Please enter the city name: ")
        state_code = input("Please enter the state acronym: ")
        # Initializes the API link using the user's city/state inputs and 
        # your API key.
        # The limit is set to only allow for one city to show up.
        city_url = f"http://api.openweathermap.org/geo/1.0/direct?q=" \
                   f"{city_name},{state_code},US&limit=1&appid={appid}"
        # Begins a try/except clause where it tries to retrieve the API link
        # information or prints out the error that occurs.
        try:
            city_info = requests.get(city_url)
            city_info.raise_for_status()
        # Handles invalid URL inputs.
        except requests.exceptions.HTTPError:
            print("\nThere was an HTTPError. Please make sure you entered a "
                  "valid city and state and try again.")
        # Handles internet connection issues.
        except requests.exceptions.ConnectionError:
            print("\nThere was an error connecting to the API. Please make "
                  "sure your internet connection is secure.")
        # Handles other, unexpected errors and prints the error.
        except Exception as error: 
            print(f"Sorry there was an unexpected error: {error}")
        # If the get request was successful, it prints off a success message
        # and passes the information on to parse_json() to parse.
        else:
            if city_info.status_code == 200:
                print("\nThe connection to the latitude/longitude information "
                      "was successful.")
                try:
                    city_data = city_info.json()[0]  # To get rid of brackets.
                    parse_json(city_data, is_zip=False)  # is_zip is False to
                    # skip the reverse geocoding in the parse_json() function.
                # Handles any invalid inputs for city/state.
                except IndexError:
                    print("It seems you did not input a correct city or state. "
                          "Please try again and input a valid response.")
            # To handle any unexpected error messages.
            else:
                print("I'm sorry. There was an unexpected error.")

In [6]:
# This function is to call for the zip code API link based on the user's zip code.
def zip_code_func(): 
    # Loops through the following input until an acceptable response is received.
    while True:
        # Asks the user to input a zip code.
        zip_code = input("Please enter the zip code: ")
        # If the length of the zip code does not equal 5 characters, it tells
        # the user to input a valid zip code.
        if len(zip_code) != 5:
            print("Please enter a valid zip code.")
        # Initializes the zip code API URL with the user's input and your API
        # key. The limit is set to only allow one city to show up.
        zip_url = f"http://api.openweathermap.org/geo/1.0/zip?zip={zip_code}," \
                  f"US&appid={appid}"
        # Begins a try/except clause where it requests the zip code API link
        # and checks for errors.
        try:
            zip_info = requests.get(zip_url)
            zip_info.raise_for_status()
        # Handles invalid URL inputs.
        except requests.exceptions.HTTPError:  
            print("\nSorry, there was an HTTPError. Please enter a valid zip "
                  "code.")
        # Handles internet connection issues.
        except requests.exceptions.ConnectionError: 
            print("\nThere was an error connecting to the API. Please make "
                  "sure your internet connection is secure and try again.")
        # Handles other, unexpected errors and prints it out.
        except Exception as error:  
            print(f"Sorry there was an unexpected error: {error}")
        # If the connection was successful, it prints out a success message
        # and passes the information along to parse_json() to parse.
        else:
            if zip_info.status_code == 200:
                print("\nThe connection to the latitude/longitude information "
                      "was successful.")
                zip_data = zip_info.json()
                parse_json(zip_data, is_zip=True)  # is_zip is True to reverse
                # geocode in the parse_json() function to get state info.
            # Handles other, unexpected errors.
            else:
                print("I'm sorry. There was an unexpected error.")

In [7]:
# This function is to parse through the retrieved JSON information.
def parse_json(json_data, is_zip):  
    # Finds the latitude and longitude of the user's input.
    lat = json_data['lat']
    lon = json_data['lon']
    #  This is to reverse geocode the zip code information since there is no 
    # "state" information on the initial zip code info retrieval.
    if is_zip is True:  
        reverse_geocode = f"http://api.openweathermap.org/geo/1.0/reverse?lat" \
                          f"={lat}&lon={lon}&limit=1&appid={appid}"
        try:
            get_reverse_info = requests.get(reverse_geocode)
            get_reverse_info.raise_for_status()
        # Handles any invalid inputs.
        except requests.exceptions.HTTPError: 
            print("\nSorry, there was an HTTPError. Please make sure your "
                "internet connection is secure and try again.")
        # Handles any internet connection issues.
        except requests.exceptions.ConnectionError: 
            print("\nThere was an error connecting to the API. Please try "
                  "again.")
        # Handles other, unexpected errors and prints them.
        except Exception as error: 
            print(f"Sorry there was an unexpected error: {error}")
        # If the connection status is successful, it prints out a success 
        # message.
        else:
            if get_reverse_info.status_code == 200:
                print("Retrieval of zip code city and state information was "
                      "successful.")
                # Grabs the first element of the JSON data and retrieves the 
                # name and state variables.
                zip_city_state = get_reverse_info.json()[0]
                city = zip_city_state['name']
                state = zip_city_state['state']
    # If the user input city/state info, it will retrieve the name and state
    # variables from the JSON data.
    else:  
        city = json_data['name']
        state = json_data['state']
    # Passes along the latitude, longitude, city, and state to get_weather().
    get_weather(lat, lon, city, state)

In [8]:
# This function is to retrieve the current weather information and to get the 
# user's preference for the temperature format.
def get_weather(lat, lon, city, state): 
    # Loops through the following code until an accepted input is received.
    while True:
        # Asks the user if they would like the temperature to be displayed in
        # Fahrenheit, Celsius, or Kelvin.
        try:
            temp_unit = input("\nWhat temperature format would you like? F - "
                              "Fahrenheit; C - Celsius; K - Kelvin: ")
            # If they say f/F (Fahrenheit), the temperature unit will be 
            # imperial.
            if temp_unit.lower() == "f":
                temp_unit = 'imperial'
            # If they say c/C (Celsius), the temperature unit will be metric.
            elif temp_unit.lower() == "c":
                temp_unit = 'metric'
            # If they say k/K (Kelvin), the temperature unit will be standard.
            elif temp_unit.lower() == "k":
                temp_unit = 'standard'
            # Handles other inputs besides f, c, and k.
            else:
                print("Please enter an 'F', 'C', or 'K' for the format.")
        # Handles other inputs besides f, c, and k.
        except ValueError:
            print("Please only enter 'F', 'C', or 'K' for the temperature.")
        # Breaks the loop if it encounters anything else.
        else:
            break
    # Initializes the weather API URL with the latitude, longitude, API key, and
    # temperature unit.
    weather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}" \
                  f"&lon={lon}&appid={appid}&units={temp_unit}"
    # Begins a try/except clause where it tries to retrieve the weather information
    # from the weather API link.
    try:
        weather_info = requests.get(weather_url)
        weather_info.raise_for_status()
    # Handles any invalid inputs.
    except requests.exceptions.HTTPError: 
        print("\nSorry, there was an HTTPError. Please make sure your internet "
              "connection is secure and try again.")
    # Handles any internet connection issues.
    except requests.exceptions.ConnectionError: 
        print("\nThere was an error connecting to the API. Please try again.")
    # Handles any other, unexpected errors and prints them.
    except Exception as error:
        print(f"Sorry there was an unexpected error: {error}")
    # If the connection was successful, it prints a success message.
    else:
        if weather_info.status_code == 200:
            print("\nThe connection to the weather information was successful.")
            # Stores the weather information into 'weather_data'.
            weather_data = weather_info.json()
            # Sends the information to parse_weather()
            parse_weather(weather_data, city, state)
        # Handles any unexpected errors.
        else:
            print("Sorry, there was an unexpected error.")

In [9]:
# Parses through the retrieved weather data to find the variables of interest.
def parse_weather(weather, city, state):  
    # Extracts the weather description.
    weather_description = weather['weather'][0]['description']
    # Extracts the current temperature.
    current_temp = weather['main']['temp']
    # Extracts the 'feels like' temperature.
    feels_like_temp = weather['main']['feels_like']
    # Extracts the minimum temperature for that area.
    temp_min = weather['main']['temp_min']
    # Extracts the maximum temperature for that area.
    temp_max = weather['main']['temp_max']
    # Extracts the pressure.
    pressure = weather['main']['pressure']
    # Extracts the humidity.
    humidity = weather['main']['humidity']
    # Sends all the extracted information to pretty_print().
    pretty_print(city, state, weather_description, current_temp,
                 feels_like_temp, temp_min, temp_max, pressure, humidity)

In [10]:
# Prints out all the extracted information in an easy-to-read format for the
# user.
def pretty_print(city, state, description, current, feels_like, min_temp,
                 max_temp, pressure, humidity):  
    # Adds a line of hyphens to break up the space.
    line_breaker = "-" * 50
    # Prints out all of the information in an easy-to-read manner.
    print(f"\n{line_breaker}\nCurrent Weather for {city}, {state}"
          f"\n{line_breaker}")
    print(f"Weather Description: {description.title()}")
    print(f"Current Temperature in Your Chosen Unit: {current}°")
    print(f"Feels Like: {feels_like}°")
    print(f"Minimum Temperature in the Area: {min_temp}°")
    print(f"Maximum Temperature in the Area: {max_temp}°")
    print(f"Pressure: {pressure} hPa")
    print(f"Humidity: {humidity}%")
    # Moves along to the use_again() function.
    use_again()

In [11]:
# This function asks the user if they would like to use the generator again.
def use_again():  
    # Loops through the following code until a valid input is received.
    while True:
        # Asks the user if they would like to use the weather service again.
        perform_again = input("\nWould you like to use the weather service "
                              "again? Y - Yes; N - No: ")
        # If they say yes, it moves back to the retrieve_user_info() function.
        if perform_again.lower() == "y":
            retrieve_user_info()
        # If they say no, it prints out a thank you message and exits.
        elif perform_again.lower() == "n":
            print("Thank you for using this program!")
            exit()
        # If an invalid response was received, it asks the user to say 
        # yes or no.
        else:
            print("Try again and only enter 'Y' or 'N'.")

#### Using the Generator

In [12]:
# This runs the entire program if the script is being ran directly and
# was not imported.
if __name__ == "__main__":
    main()

Welcome to the United States OpenWeather Generator! Please keep in mind to only enter U.S. cities, states, or zip codes.

Would you like to use the weather service? Y - Yes; N - No: n
Thanks for using the program!
